<a href="https://colab.research.google.com/github/zwubbena/sppi_3b_multi_level_analysis/blob/main/sppi_3b_multi_level_region_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
================================================================================
SPPI 3B Multi-Level Comparative Analysis for Texas Reading Proficiency - State vs. Region
================================================================================

Program: SPPI 3B Multi-Level Comparative Analysis for Texas Reading Proficiency:
  State vs. Education Service Center (ESC) Region
  (comparative multi-region analysis framework)
Author: Zane Wubbena, PhD
Version: 4.0
Date: May 22, 2025
Last Updated: July 10, 2025
Platform: Python, Google Colab

Description:
This script generates comprehensive multi-level comparative analysis Excel
workbooks that examine SPPI 3B reading proficiency rates among students with
IEPs across all 20 Education Service Center (ESC) regions in Texas compared
to statewide performance. The analysis is based on State of Texas Assessments
of Academic Readiness (STAAR) for English Language Arts (ELA) in grades 4 and 8
and English I in High School from SY 2020-21 to SY 2023-24.

Analysis Levels:
- Level 1: Total proficiency rates by individual grade level
- Level 2: Total proficiency rates for all grades combined
- Level 3: Accommodation status breakdown by individual grade level
- Level 4: Accommodation status breakdown for all grades combined

Input: CSV file containing SPPI 3B assessment data
Output: 20 Excel workbooks (one per region) with data tables, trend graphs,
        and professional formatting

Features:
- Professional styling with consistent color scheme and fonts
- Interactive data tables with readable variable labels
- Trend graphs with data point annotations and state targets
- Comprehensive data dictionary and ReadMe documentation
- Footnotes in styled boxes under data tables
- High-resolution graph exports (300 DPI)

Requirements:
- pandas, matplotlib, openpyxl libraries
- CSV data file with specified column structure
- Google Colab environment

Usage:
1. Upload CSV data file when prompted
2. Script automatically processes all 20 regions
3. Downloads zip file containing all Excel workbooks and graphs

================================================================================
"""

!pip install openpyxl matplotlib

import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import textwrap
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
from google.colab import files

# Upload the file
uploaded = files.upload()
filename = next(iter(uploaded))
df = pd.read_csv(filename)

# Variable labels for display
variable_labels = {
    "YEAR_FLAG": "Determination Year",
    "FFY": "Federal Fiscal Year",
    "SCHOOL_YEAR": "School Year",
    "GROUP": "Administrative Level",
    "GRADE_LEVEL": "Grade Level",
    "SUBJECT": "Subject Area",
    "TOT_IEP_CNT": "Total IEP Students",
    "TOT_PROFICIENT_CNT": "Total Proficient Students",
    "TOT_PROFICIENT_PCT": "Total Proficient (%)",
    "R_WITH_CNT": "Regular STAAR with Accommodations (Count)",
    "R_WITH_PCT": "Regular STAAR with Accommodations (%)",
    "A_WITH_CNT": "Advanced STAAR with Accommodations (Count)",
    "A_WITH_PCT": "Advanced STAAR with Accommodations (%)",
    "R_WOUT_CNT": "Regular STAAR without Accommodations (Count)",
    "R_WOUT_PCT": "Regular STAAR without Accommodations (%)",
    "A_WOUT_CNT": "Advanced STAAR without Accommodations (Count)",
    "A_WOUT_PCT": "Advanced STAAR without Accommodations (%)",
    "TOT_RA_WITH_CNT": "Total Proficient with Accommodations (Count)",
    "TOT_RA_WITH_PCT": "Total Proficient with Accommodations (%)",
    "TOT_RA_WOUT_CNT": "Total Proficient without Accommodations (Count)",
    "TOT_RA_WOUT_PCT": "Total Proficient without Accommodations (%)",
    "DATE_PROCESSED": "Processing Date"
}

# Footnotes for each Level type
level_footnotes = {
    "Level1": ("Note: Percentages represent weighted averages combining data from each grade level "
               "(Grade 4, Grade 8, and High School) over a four-year period. Calculations are performed "
               "separately for regional and state levels. The percentages are calculated as the total "
               "number of students proficient at grade level or above (numerator) divided by the total "
               "number of students with Individualized Education Programs (IEP) who have valid assessment "
               "results (denominator)."),

    "Level2": ("Note: Percentages represent weighted averages combining data from all three grade levels "
               "(Grade 4, Grade 8, and High School) over a four-year period. Calculations are performed "
               "separately for regional and state levels. The percentages are calculated as the total "
               "number of students proficient at grade level or above (numerator) divided by the total "
               "number of students with Individualized Education Programs (IEP) who have valid assessment "
               "results (denominator)."),

    "Level3": ("Note: Percentages represent weighted averages combining data from each grade level "
               "(Grade 4, Grade 8, and High School) over a four-year period. Two distinct percentages "
               "are reported for each level: the percentage of students proficient with accommodations, "
               "calculated as total students proficient with accommodations (numerator) divided by total "
               "students proficient at grade level or above (denominator), and the percentage of students "
               "proficient without accommodations, calculated as total students proficient without "
               "accommodations (numerator) divided by total students proficient at grade level or above "
               "(denominator)."),

    "Level4": ("Note: Percentages represent weighted averages combining data from all three grade levels "
               "(Grade 4, Grade 8, and High School) over a four-year period. Calculations are performed "
               "separately for regional and state levels. Two distinct percentages are reported for each "
               "level: the percentage of students proficient with accommodations, calculated as total "
               "students proficient with accommodations (numerator) divided by total students proficient "
               "at grade level or above (denominator), and the percentage of students proficient without "
               "accommodations, calculated as total students proficient without accommodations (numerator) "
               "divided by total students proficient at grade level or above (denominator).")
}

# State Target values
# This Python snippet defines a dictionary named state_targets that maps school years to numeric target values.
# Here's what it does:
# Each key in the dictionary (like "SY 2021-22") represents a school year.
# Each value (like 12.0) represents the state target for that year
# The dictionary allows you to easily look up or iterate over targets by year.
state_targets = {
    "Grade 4": {
        "SY 2020-21": 11.8,
        "SY 2021-22": 14.0,
        "SY 2022-23": 20.0,
        "SY 2023-24": 25.0,
    },
    "Grade 8": {
        "SY 2020-21": 10.0,
        "SY 2021-22": 12.0,
        "SY 2022-23": 18.0,
        "SY 2023-24": 25.0,
    },
    "High School": {
        "SY 2020-21": 10.5,
        "SY 2021-22": 12.0,
        "SY 2022-23": 18.0,
        "SY 2023-24": 25.0,
    }
}

# Professional styling definitions
def apply_professional_styling():
    # Color scheme
    colors = {
        'primary_header': 'D5E8F7',      # Light blue
        'secondary_header': 'E8F4FD',    # Very light blue
        'accent': '2F75B5',              # Dark blue
        'text_dark': '1F497D',           # Dark blue text
        'border_color': '4F81BD',        # Medium blue border
        'note_background': 'F8F9FA'      # Light gray for note boxes
    }

    # Fonts
    fonts = {
        'title': Font(name='Calibri', size=14, bold=True, color=colors['accent']),
        'header': Font(name='Calibri', size=11, bold=True, color=colors['text_dark']),
        'subheader': Font(name='Calibri', size=10, bold=True, color=colors['text_dark']),
        'body': Font(name='Calibri', size=10),
        'note': Font(name='Calibri', size=9, italic=True, color='666666')
    }

    # Fills
    fills = {
        'primary_header': PatternFill(start_color=colors['primary_header'],
                                     end_color=colors['primary_header'],
                                     fill_type='solid'),
        'secondary_header': PatternFill(start_color=colors['secondary_header'],
                                       end_color=colors['secondary_header'],
                                       fill_type='solid'),
        'note_box': PatternFill(start_color=colors['note_background'],
                               end_color=colors['note_background'],
                               fill_type='solid')
    }

    # Borders
    thin_border = Border(
        left=Side(style='thin', color=colors['border_color']),
        right=Side(style='thin', color=colors['border_color']),
        top=Side(style='thin', color=colors['border_color']),
        bottom=Side(style='thin', color=colors['border_color'])
    )

    # Note box border (thicker for emphasis)
    note_border = Border(
        left=Side(style='medium', color=colors['border_color']),
        right=Side(style='medium', color=colors['border_color']),
        top=Side(style='medium', color=colors['border_color']),
        bottom=Side(style='medium', color=colors['border_color'])
    )

    return colors, fonts, fills, thin_border, note_border

# Function to format headers professionally
def format_professional_header(sheet, row_num, col_start, col_end, fonts, fills, border):
    for col in range(col_start, col_end + 1):
        cell = sheet.cell(row=row_num, column=col)
        cell.font = fonts['header']
        cell.fill = fills['primary_header']
        cell.border = border
        cell.alignment = Alignment(horizontal='center', vertical='center')

# Function to format data rows professionally
def format_data_rows(sheet, start_row, end_row, col_start, col_end, fonts, border):
    for row in range(start_row, end_row + 1):
        for col in range(col_start, col_end + 1):
            cell = sheet.cell(row=row, column=col)
            cell.font = fonts['body']
            cell.border = border
            cell.alignment = Alignment(horizontal='center', vertical='center')

# Function to add footnote in a styled box
def add_footnote_box(sheet, footnote_text, start_row, num_columns, fonts, fills, border):
    # Add the footnote text
    sheet.cell(row=start_row, column=1, value=footnote_text)

    # Merge cells for the footnote
    end_column_letter = get_column_letter(num_columns)
    sheet.merge_cells(f'A{start_row}:{end_column_letter}{start_row}')

    # Style the footnote cell
    footnote_cell = sheet.cell(row=start_row, column=1)
    footnote_cell.font = fonts['note']
    footnote_cell.fill = fills['note_box']
    footnote_cell.border = border
    footnote_cell.alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')

    # Set row height for better visibility
    sheet.row_dimensions[start_row].height = 60

# Standardize fields
df['GROUP'] = df['GROUP'].astype(str).str.strip().str.upper()
df['GRADE_LEVEL'] = df['GRADE_LEVEL'].astype(str).str.strip().str.upper()
df['SCHOOL_YEAR'] = df['SCHOOL_YEAR'].astype(str).str.strip()

# Filters
target_years = ['SY 2020-21', 'SY 2021-22', 'SY 2022-23', 'SY 2023-24']
grade_levels = {
    'GRADE 04': 'Grade 4',
    'GRADE 08': 'Grade 8',
    'GRADE HS': 'High School'
}
year_range_text = "SY 2020–21 to SY 2023–24"

# Create main output folder and graphs subfolder
output_dir = "/content/sppi_3b_multi_level_analysis"
graphs_dir = f"{output_dir}/graphs"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(graphs_dir, exist_ok=True)

for i in range(1, 21):
    region_code = f"{i:02d}"
    region_group = f"REGION {region_code}"
    region_label = f"Region {region_code}"

    wb = Workbook()
    wb.remove(wb.active)

    # Get professional styling
    colors, fonts, fills, thin_border, note_border = apply_professional_styling()

    # === Create README sheet first ===
    readme_sheet = wb.create_sheet("README")

    # Set column width
    readme_sheet.column_dimensions['A'].width = 20
    readme_sheet.column_dimensions['B'].width = 100

    # Title
    readme_sheet.append(["README", ""])
    readme_sheet.merge_cells('A1:B1')
    title_cell = readme_sheet['A1']
    title_cell.font = fonts['title']
    title_cell.fill = fills['primary_header']
    title_cell.alignment = Alignment(horizontal='left')

    # Introduction
    readme_sheet.append(["", ""])
    readme_sheet.append([
        "Introduction",
        f"""This Excel workbook presents a multi-level comparative analysis of State Performance Plan Indicator (SPPI) 3B, examining reading proficiency rates among students with IEPs in {region_label} compared to statewide results. The analysis uses State of Texas Assessments of Academic Readiness (STAAR) data for English Language Arts (ELA) in grades 4 and 8, and English I at the high school level, covering school years {year_range_text}.

        The workbook is structured into four analytical levels:
        - Level 1: Proficiency rates by individual grade
        - Level 2: Combined proficiency rates across all grades
        - Level 3: Grade-level breakdown of proficiency rates by accommodation status (with vs. without accommodations)
        - Level 4: Combined proficiency rates by accommodation status across all grades

        This structure offers a clear progression for understanding reading proficiency among students with IEPs in {region_label}, as well as how these outcomes compare to the state. Stakeholders can first examine overall proficiency rates, then explore how accommodation status affects proficiency results. The design supports a step-by-step interpretation of trends and performance gaps between {region_label} and the state."""
    ])
    readme_sheet['A3'].font = fonts['subheader']
    readme_sheet['A3'].fill = fills['secondary_header']

    # Worksheet descriptions
    readme_sheet.append(["", ""])
    readme_sheet.append(["Worksheet Description", ""])
    readme_sheet['A5'].font = fonts['subheader']
    readme_sheet['A5'].fill = fills['secondary_header']

    sheet_descriptions = [
        ["Data", f"This worksheet contains the raw assessment data for students with IEPs in {region_label} and statewide, including proficiency counts and accommodation status (with and without accommodations) across multiple school years."],
        ["Data Dictionary", "This worksheet provides definitions and explanations for all variables used in the Data worksheet."],
        ["Level 1 Grade 4", f"This worksheet compares Grade 4 ELA proficiency rates between {region_label} and the state for students with IEPs. It includes a data table and trend graph showing proficiency rates over {year_range_text}."],
        ["Level 1 Grade 8", f"This worksheet compares Grade 8 ELA proficiency rates between {region_label} and the state for students with IEPs. It includes a data table and trend graph showing proficiency rates over {year_range_text}."],
        ["Level 1 High School", f"This worksheet compares High School English I proficiency rates between {region_label} and the state for students with IEPs. It includes a data table and trend graph showing proficiency rates over {year_range_text}."],
        ["Level 2 Combined", f"This worksheet combines data from all grade levels (Grades 4, 8, and High School) to compare overall proficiency rates between {region_label} and the state for students with IEPs. It includes a data table and trend graph showing combined proficiency trends over {year_range_text}."],
        ["Level 3 Grade 4", f"This worksheet analyzes Grade 4 students with IEPs who achieved proficiency, showing the breakdown of those who succeeded with accommodations versus those who succeeded without accommodations. It compares {region_label} and state performance over {year_range_text}."],
        ["Level 3 Grade 8", f"This worksheet analyzes Grade 8 students with IEPs who achieved proficiency, showing the breakdown of those who succeeded with accommodations versus those who succeeded without accommodations. It compares {region_label} and state performance over {year_range_text}."],
        ["Level 3 High School", f"This worksheet analyzes High School English I students with IEPs who achieved proficiency, showing the breakdown of those who succeeded with accommodations versus those who succeeded without accommodations. It compares {region_label} and state performance over {year_range_text}."],
        ["Level 4 Combined", f"This worksheet combines data from all grade levels to analyze students with IEPs who achieved proficiency, showing the breakdown of those who succeeded with accommodations versus those who succeeded without accommodations. It compares {region_label} and state performance over {year_range_text}."]
    ]

    for desc in sheet_descriptions:
        readme_sheet.append(desc)
        readme_sheet.cell(row=readme_sheet.max_row, column=1).font = fonts['subheader']

    # Notes and terminology
    readme_sheet.append(["", ""])
    readme_sheet.append(["Key Terminology", ""])
    # Use the current row number instead of hardcoded A5
    current_row = readme_sheet.max_row
    readme_sheet.cell(row=current_row, column=1).font = fonts['subheader']
    readme_sheet.cell(row=current_row, column=1).fill = fills['secondary_header']

    terminology = [
        ["ELA", "English Language Arts"],
        ["EOC", "End of Course"],
        ["IEP", "Individualized Education Program - Students with disabilities who receive special education services with IEPs."],
        ["Level 1 & 2", "Y-Axis Label - % of IEP Students Proficient; What It Measures - Out of all IEP students, what % are proficient"],
        ["Level 3 & 4", "Y-Axis Label - % of Proficient Students with IEPs; What It Measures - Out of proficient IEP students, what % used/didn't use accommodations"],
        ["Proficient", "Students who have met or exceeded the grade-level standards on the statewide assessment."],
        ["STAAR", "State of Texas Assessments of Academic Readiness"],
        ["With Accommodations", "Students with IEPs who used testing accommodations and who met or exceeded the grade-level standards on the statewide assessment."],
        ["Without Accommodations", "Students with IEPs who did not use testing accommodations and who met or exceeded the grade-level standards on the statewide assessment."],
        ["Accommodations", "A student is considered to have received accommodations if they have a value of 1 = Yes for both the Special-Education-Code variable and the Accommodation(s) variable on the English Language Arts (ELA) State of Texas Assessments of Academic Readiness (STAAR®) for Grade 4, Grade 8, or English I (End-of-Course at the high school level). These data come from the Consolidated Accountability File (CAF), which combines all STAAR, STAAR Spanish, STAAR Alternate 2, TELPAS, and TELPAS Alternate records into a single student-level file for the current accountability year. The CAF is the official source for student assessment and accountability reporting in Texas. Source: https://tea.texas.gov/texas-schools/accountability/academic-accountability/performance-reporting/2024-consolidated-accountability-file-data-file-format.pdf."]
    ]

    for term in terminology:
        readme_sheet.append(term)
        readme_sheet.cell(row=readme_sheet.max_row, column=1).font = Font(name='Calibri', size=10, bold=True, italic=True, color=colors['text_dark'])

    # Set alignment for all description cells
    for row in readme_sheet.iter_rows(min_row=2):
        if row[1].value:
            row[1].alignment = Alignment(wrap_text=True, vertical='top')
            row[1].font = fonts['body']

    # === Create Data Dictionary sheet ===
    dict_sheet = wb.create_sheet("Data Dictionary")

    # Set column widths
    dict_sheet.column_dimensions['A'].width = 18
    dict_sheet.column_dimensions['B'].width = 35
    dict_sheet.column_dimensions['C'].width = 80

    # Title
    dict_sheet.append(["Data Dictionary", "", ""])
    dict_sheet.merge_cells('A1:C1')
    title_cell = dict_sheet['A1']
    title_cell.font = fonts['title']
    title_cell.fill = fills['primary_header']
    title_cell.alignment = Alignment(horizontal='left')

    # Header
    dict_sheet.append(["", "", ""])
    dict_sheet.append(["Variable", "Label", "Description"])
    format_professional_header(dict_sheet, 3, 1, 3, fonts, fills, thin_border)

    # Add dictionary entries
    dictionary_entries = [
        ["YEAR_FLAG", "The year the State Performance Plan/Annual Performance Report (SPP/APR) determination was issued for Texas (e.g., 2025)."],
        ["FFY", "The Federal Fiscal Year (FFY) associated with the SPP/APR submission that resulted in the state determination (e.g., YEAR_FLAG = \"2025\" is associated with FFY 2023)."],
        ["SCHOOL_YEAR", "The school year from which data were primarily collected for the SPP/APR submission (e.g., YEAR_FLAG = \"2025\" corresponds to FFY 2023 and SCHOOL_YEAR = \"SY 2023-24\")."],
        ["GROUP", "The specific Education Service Center (ESC) region or statewide level being reported."],
        ["GRADE_LEVEL", "The three grade levels reported for State Performance Plan Indicator (SPPI) 3B: Grade 4, Grade 8, and High School (HS)."],
        ["SUBJECT", "The subject area tested. Grades 4 and 8 are tested in English Language Arts (ELA), while High School students take English I."],
        ["TOT_IEP_CNT", "Total count of students with IEPs who participated in the State of Texas Assessments of Academic Readiness (STAAR)."],
        ["TOT_PROFICIENT_CNT", "Total count of students with IEPs who scored at or above grade level on STAAR."],
        ["TOT_PROFICIENT_PCT", "Total percentage of students with IEPs who scored at or above grade level on STAAR."],
        ["R_WITH_CNT", "Count of students with IEPs using accommodations who scored at or above grade level on the regular STAAR assessment."],
        ["R_WITH_PCT", "Percentage of students with IEPs using accommodations who scored at or above grade level on the regular STAAR assessment."],
        ["A_WITH_CNT", "Count of students with IEPs using accommodations who scored at or above grade level on the advanced STAAR assessment."],
        ["A_WITH_PCT", "Percentage of students with IEPs using accommodations who scored at or above grade level on the advanced STAAR assessment."],
        ["R_WOUT_CNT", "Count of students with IEPs without accommodations who scored at or above grade level on the regular STAAR assessment."],
        ["R_WOUT_PCT", "Percentage of students with IEPs without accommodations who scored at or above grade level on the regular STAAR assessment."],
        ["A_WOUT_CNT", "Count of students with IEPs without accommodations who scored at or above grade level on the advanced STAAR assessment."],
        ["A_WOUT_PCT", "Percentage of students with IEPs without accommodations who scored at or above grade level on the advanced STAAR assessment."],
        ["TOT_RA_WITH_CNT", "Total count of students with IEPs using accommodations who scored at or above grade level on either regular or advanced STAAR assessments."],
        ["TOT_RA_WITH_PCT", "Total percentage of students with IEPs using accommodations who scored at or above grade level on either regular or advanced STAAR assessments."],
        ["TOT_RA_WOUT_CNT", "Total count of students with IEPs without accommodations who scored at or above grade level on either regular or advanced STAAR assessments."],
        ["TOT_RA_WOUT_PCT", "Total percentage of students with IEPs without accommodations who scored at or above grade level on either regular or advanced STAAR assessments."],
        ["DATE_PROCESSED", "The date when the data were initially processed."]
    ]

    start_row = 4
    for entry in dictionary_entries:
        variable_code = entry[0]
        description = entry[1]
        readable_label = variable_labels.get(variable_code, variable_code)

        # Add row with: Variable Code | Readable Label | Description
        dict_sheet.append([variable_code, readable_label, description])

        # Format the row
        current_row = dict_sheet.max_row
        dict_sheet.cell(row=current_row, column=1).font = fonts['subheader']
        dict_sheet.cell(row=current_row, column=2).font = Font(name='Calibri', size=10, italic=True, color=colors['text_dark'])
        dict_sheet.cell(row=current_row, column=3).font = fonts['body']

        # Apply borders
        for col in range(1, 4):
            dict_sheet.cell(row=current_row, column=col).border = thin_border

    # Set alignment for all description cells
    for row in dict_sheet.iter_rows(min_row=4):
        if row[2].value:
            row[2].alignment = Alignment(wrap_text=True, vertical='top')

    # Add Data sheet - containing both region and state data for comparison
    combined_data = df[(df['GROUP'] == region_group) | (df['GROUP'] == 'STATEWIDE')].copy()

    # Create Data sheet with readable column headers
    data_sheet = wb.create_sheet("Data")

    # First add readable column headers
    original_headers = list(combined_data.columns)
    readable_headers = [variable_labels.get(col, col) for col in original_headers]
    data_sheet.append(readable_headers)

    # Format headers
    format_professional_header(data_sheet, 1, 1, len(readable_headers), fonts, fills, thin_border)

    # Then add all data rows
    for _, row in combined_data.iterrows():
        data_sheet.append(row.tolist())

    # Format data rows
    if len(combined_data) > 0:
        format_data_rows(data_sheet, 2, len(combined_data) + 1, 1, len(readable_headers), fonts, thin_border)

    # Auto-adjust column widths
    for col in range(1, len(readable_headers) + 1):
        data_sheet.column_dimensions[get_column_letter(col)].width = 15

    # === Level 1: Graphs by Grade ===
    for grade_code, grade_label in grade_levels.items():
        r = df[(df['GROUP'] == region_group) & (df['GRADE_LEVEL'] == grade_code) & df['SCHOOL_YEAR'].isin(target_years)].copy()
        s = df[(df['GROUP'] == 'STATEWIDE') & (df['GRADE_LEVEL'] == grade_code) & df['SCHOOL_YEAR'].isin(target_years)].copy()
        if r.empty or s.empty: continue
        r['PROF_PCT'] = r['TOT_PROFICIENT_CNT'] / r['TOT_IEP_CNT'] * 100
        s['PROF_PCT'] = s['TOT_PROFICIENT_CNT'] / s['TOT_IEP_CNT'] * 100

        shared_years = sorted(set(r['SCHOOL_YEAR']) & set(s['SCHOOL_YEAR']))
        if not shared_years: continue

        fig, ax = plt.subplots(figsize=(8, 5))

        # Plot region data and add value labels
        r_values = r.set_index('SCHOOL_YEAR').loc[shared_years, 'PROF_PCT']
        ax.plot(shared_years, r_values, marker='o', label=region_label)
        for i_val, value in enumerate(r_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,10),
                        ha='center',
                        fontsize=8)

        # Plot statewide data and add value labels
        s_values = s.set_index('SCHOOL_YEAR').loc[shared_years, 'PROF_PCT']
        ax.plot(shared_years, s_values, marker='o', label='Statewide')
        for i_val, value in enumerate(s_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,-15),
                        ha='center',
                        fontsize=8)

        # Plot State Target line
        target_years_available = [yr for yr in shared_years if grade_label in state_targets and yr in state_targets[grade_label]]
        if target_years_available:
            target_values = [state_targets[grade_label][yr] for yr in target_years_available]
            ax.plot(target_years_available, target_values, marker='s', label='State Target',
                   linestyle='--', color='red')
            for i_val, value in enumerate(target_values):
                ax.annotate(f"{value:.1f}",
                            (target_years_available[i_val], value),
                            textcoords="offset points",
                            xytext=(10,0),
                            ha='left',
                            fontsize=8)

        ax.set_title(f"Level 1: {grade_label} - % Proficient\n{region_label} vs. Statewide ({year_range_text})")
        ax.set_xlabel("School Year")
        ax.set_ylabel("% of IEP Students Proficient")
        ax.set_ylim(0, 40)
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        # Save to graphs directory (no footnote on graph)
        img_filename = f"{region_label.lower().replace(' ', '_')}_level1_{grade_label.lower().replace(' ', '_')}.png"
        img_path = f"{graphs_dir}/{img_filename}"
        fig.savefig(img_path, dpi=300, bbox_inches='tight')
        plt.close()

        sheet = wb.create_sheet(f"Level 1 {grade_label}")
        headers = [variable_labels.get("SCHOOL_YEAR", "School Year"),
                  f"{region_label} Proficient (%)",
                  "Statewide Proficient (%)",
                  "State Target (%)"]
        sheet.append(headers)

        # Format headers professionally
        format_professional_header(sheet, 1, 1, len(headers), fonts, fills, thin_border)

        for yr in shared_years:
            target_val = ""
            if grade_label in state_targets and yr in state_targets[grade_label]:
                target_val = state_targets[grade_label][yr]

            sheet.append([
                yr,
                round(r[r['SCHOOL_YEAR'] == yr]['PROF_PCT'].values[0], 1),
                round(s[s['SCHOOL_YEAR'] == yr]['PROF_PCT'].values[0], 1),
                target_val if target_val != "" else ""
            ])

        # Format data rows professionally
        format_data_rows(sheet, 2, len(shared_years) + 1, 1, len(headers), fonts, thin_border)

        # Auto-adjust column widths
        for col in range(1, len(headers) + 1):
            sheet.column_dimensions[get_column_letter(col)].width = 20

        # Add image with Level 1 specific sizing: Height: 4.86" (467px) Width: 7.85" (754px)
        img = XLImage(img_path)
        img.width = 754
        img.height = 467
        sheet.add_image(img, "A11")

        # Add Level1 footnote in a styled box below the data table
        footnote_row = len(shared_years) + 3  # Add some space below the data
        add_footnote_box(sheet, level_footnotes["Level1"], footnote_row, len(headers), fonts, fills, note_border)

    # === Level 2: All Grades Combined ===
    r = df[(df['GROUP'] == region_group) & df['GRADE_LEVEL'].isin(grade_levels.keys()) & df['SCHOOL_YEAR'].isin(target_years)].copy()
    s = df[(df['GROUP'] == 'STATEWIDE') & df['GRADE_LEVEL'].isin(grade_levels.keys()) & df['SCHOOL_YEAR'].isin(target_years)].copy()

    if not r.empty and not s.empty:
        r_sum = r.groupby('SCHOOL_YEAR')[['TOT_PROFICIENT_CNT','TOT_IEP_CNT']].sum()
        s_sum = s.groupby('SCHOOL_YEAR')[['TOT_PROFICIENT_CNT','TOT_IEP_CNT']].sum()
        r_sum['PROF_PCT'] = r_sum['TOT_PROFICIENT_CNT'] / r_sum['TOT_IEP_CNT'] * 100
        s_sum['PROF_PCT'] = s_sum['TOT_PROFICIENT_CNT'] / s_sum['TOT_IEP_CNT'] * 100
        shared_years = sorted(set(r_sum.index) & set(s_sum.index))

        fig, ax = plt.subplots(figsize=(8, 5))

        # Plot region data and add value labels
        r_values = r_sum.loc[shared_years, 'PROF_PCT']
        ax.plot(shared_years, r_values, marker='o', label=region_label)
        for i_val, value in enumerate(r_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,10),
                        ha='center',
                        fontsize=8)

        # Plot statewide data and add value labels
        s_values = s_sum.loc[shared_years, 'PROF_PCT']
        ax.plot(shared_years, s_values, marker='o', label="Statewide")
        for i_val, value in enumerate(s_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,-15),
                        ha='center',
                        fontsize=8)

        # Plot averaged State Target line
        target_years_available = [yr for yr in shared_years if any(yr in state_targets[grade] for grade in state_targets)]
        if target_years_available:
            target_values = []
            for yr in target_years_available:
                vals = [state_targets[grade][yr] for grade in state_targets if yr in state_targets[grade]]
                target_values.append(sum(vals) / len(vals))

            ax.plot(target_years_available, target_values, marker='s', label='Avg State Target',
                   linestyle='--', color='red')
            for i_val, value in enumerate(target_values):
                ax.annotate(f"{value:.1f}",
                            (target_years_available[i_val], value),
                            textcoords="offset points",
                            xytext=(10,0),
                            ha='left',
                            fontsize=8)

        ax.set_title(f"Level 2: All Grades Combined - % Proficient\n{region_label} vs. Statewide ({year_range_text})")
        ax.set_xlabel("School Year")
        ax.set_ylabel("% of IEP Students Proficient")
        ax.set_ylim(0, 40)
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        # Save to graphs directory (no footnote on graph)
        img_filename = f"{region_label.lower().replace(' ', '_')}_level2_combined.png"
        img_path = f"{graphs_dir}/{img_filename}"
        fig.savefig(img_path, dpi=300, bbox_inches='tight')
        plt.close()

        # Create Excel sheet
        sheet = wb.create_sheet("Level 2 Combined")
        headers = [variable_labels.get("SCHOOL_YEAR", "School Year"),
                  f"{region_label} Proficient (%)",
                  "Statewide Proficient (%)",
                  "State Target (%)"]
        sheet.append(headers)

        # Format headers professionally
        format_professional_header(sheet, 1, 1, len(headers), fonts, fills, thin_border)

        # Write rows with average target per year
        for yr in shared_years:
            # Compute average state target across all grades for this year (if available)
            targets = [state_targets[grade][yr] for grade in state_targets if yr in state_targets[grade]]
            target_val = round(sum(targets) / len(targets), 1) if targets else ""

            sheet.append([
                yr,
                round(r_sum.loc[yr, 'PROF_PCT'], 1),
                round(s_sum.loc[yr, 'PROF_PCT'], 1),
                target_val
            ])

        # Format data rows professionally
        format_data_rows(sheet, 2, len(shared_years) + 1, 1, len(headers), fonts, thin_border)

        # Auto-adjust column widths
        for col in range(1, len(headers) + 1):
            sheet.column_dimensions[get_column_letter(col)].width = 20

        # Add image with Level 2 specific sizing: Height: 4.86" (467px) Width: 7.85" (754px)
        img = XLImage(img_path)
        img.width = 754
        img.height = 467
        sheet.add_image(img, "A11")

        # Add Level2 footnote in a styled box below the data table
        footnote_row = len(shared_years) + 3  # Add some space below the data
        add_footnote_box(sheet, level_footnotes["Level2"], footnote_row, len(headers), fonts, fills, note_border)

    # === Level 3: With vs. Without Accommodations by Grade ===
    for grade_code, grade_label in grade_levels.items():
        r = df[(df['GROUP'] == region_group) & (df['GRADE_LEVEL'] == grade_code) & df['SCHOOL_YEAR'].isin(target_years)].copy()
        s = df[(df['GROUP'] == 'STATEWIDE') & (df['GRADE_LEVEL'] == grade_code) & df['SCHOOL_YEAR'].isin(target_years)].copy()
        if r.empty or s.empty: continue
        r['WITH_PCT'] = r['TOT_RA_WITH_CNT'] / r['TOT_PROFICIENT_CNT'] * 100
        r['WOUT_PCT'] = r['TOT_RA_WOUT_CNT'] / r['TOT_PROFICIENT_CNT'] * 100
        s['WITH_PCT'] = s['TOT_RA_WITH_CNT'] / s['TOT_PROFICIENT_CNT'] * 100
        s['WOUT_PCT'] = s['TOT_RA_WOUT_CNT'] / s['TOT_PROFICIENT_CNT'] * 100
        shared_years = sorted(set(r['SCHOOL_YEAR']) & set(s['SCHOOL_YEAR']))

        fig, ax = plt.subplots(figsize=(9, 5))

        # Plot region with accommodations and add value labels
        rw_values = r.set_index('SCHOOL_YEAR').loc[shared_years, 'WITH_PCT']
        ax.plot(shared_years, rw_values, marker='o', label=f'{region_label} - With')
        for i_val, value in enumerate(rw_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,10),
                        ha='center',
                        fontsize=8)

        # Plot region without accommodations and add value labels
        rwo_values = r.set_index('SCHOOL_YEAR').loc[shared_years, 'WOUT_PCT']
        ax.plot(shared_years, rwo_values, marker='o', label=f'{region_label} - Without')
        for i_val, value in enumerate(rwo_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,-15),
                        ha='center',
                        fontsize=8)

        # Plot state with accommodations and add value labels
        sw_values = s.set_index('SCHOOL_YEAR').loc[shared_years, 'WITH_PCT']
        ax.plot(shared_years, sw_values, marker='o', linestyle='--', label='State - With')
        for i_val, value in enumerate(sw_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(10,0),
                        ha='left',
                        fontsize=8)

        # Plot state without accommodations and add value labels
        swo_values = s.set_index('SCHOOL_YEAR').loc[shared_years, 'WOUT_PCT']
        ax.plot(shared_years, swo_values, marker='o', linestyle='--', label='State - Without')
        for i_val, value in enumerate(swo_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(-10,0),
                        ha='right',
                        fontsize=8)

        ax.set_title(f"Level 3: {grade_label} - Accommodation Status (With & Without)\n{region_label} vs. Statewide ({year_range_text})")
        ax.set_xlabel("School Year")
        ax.set_ylabel("% of Proficient Students with IEPs")
        ax.set_ylim(0, 100)
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        # Save to graphs directory (no footnote on graph)
        img_filename = f"{region_label.lower().replace(' ', '_')}_level3_{grade_label.lower().replace(' ', '_')}.png"
        img_path = f"{graphs_dir}/{img_filename}"
        fig.savefig(img_path, dpi=300, bbox_inches='tight')
        plt.close()

        sheet = wb.create_sheet(f"Level 3 {grade_label}")
        headers = [variable_labels.get("SCHOOL_YEAR", "School Year"),
                  f"{region_label} - With Accommodations (%)",
                  f"{region_label} - Without Accommodations (%)",
                  "State - With Accommodations (%)",
                  "State - Without Accommodations (%)"]
        sheet.append(headers)

        # Format headers professionally
        format_professional_header(sheet, 1, 1, len(headers), fonts, fills, thin_border)

        for yr in shared_years:
            sheet.append([
                yr,
                round(r[r['SCHOOL_YEAR'] == yr]['WITH_PCT'].values[0], 1),
                round(r[r['SCHOOL_YEAR'] == yr]['WOUT_PCT'].values[0], 1),
                round(s[s['SCHOOL_YEAR'] == yr]['WITH_PCT'].values[0], 1),
                round(s[s['SCHOOL_YEAR'] == yr]['WOUT_PCT'].values[0], 1)
            ])

        # Format data rows professionally
        format_data_rows(sheet, 2, len(shared_years) + 1, 1, len(headers), fonts, thin_border)

        # Auto-adjust column widths
        for col in range(1, len(headers) + 1):
            sheet.column_dimensions[get_column_letter(col)].width = 22

        # Add image with Level 3 specific sizing: Height: 4.86" (467px) Width: 8.83" (848px)
        img = XLImage(img_path)
        img.width = 848
        img.height = 467
        sheet.add_image(img, "A11")

        # Add Level3 footnote in a styled box below the data table
        footnote_row = len(shared_years) + 3  # Add some space below the data
        add_footnote_box(sheet, level_footnotes["Level3"], footnote_row, len(headers), fonts, fills, note_border)

    # === Level 4: Combined by Accommodations ===
    r = df[(df['GROUP'] == region_group) & df['GRADE_LEVEL'].isin(grade_levels.keys()) & df['SCHOOL_YEAR'].isin(target_years)].copy()
    s = df[(df['GROUP'] == 'STATEWIDE') & df['GRADE_LEVEL'].isin(grade_levels.keys()) & df['SCHOOL_YEAR'].isin(target_years)].copy()
    if not r.empty and not s.empty:
        r_sum = r.groupby('SCHOOL_YEAR')[['TOT_PROFICIENT_CNT','TOT_RA_WITH_CNT','TOT_RA_WOUT_CNT']].sum()
        r_sum['WITH_PCT'] = r_sum['TOT_RA_WITH_CNT'] / r_sum['TOT_PROFICIENT_CNT'] * 100
        r_sum['WOUT_PCT'] = r_sum['TOT_RA_WOUT_CNT'] / r_sum['TOT_PROFICIENT_CNT'] * 100
        s_sum = s.groupby('SCHOOL_YEAR')[['TOT_PROFICIENT_CNT','TOT_RA_WITH_CNT','TOT_RA_WOUT_CNT']].sum()
        s_sum['WITH_PCT'] = s_sum['TOT_RA_WITH_CNT'] / s_sum['TOT_PROFICIENT_CNT'] * 100
        s_sum['WOUT_PCT'] = s_sum['TOT_RA_WOUT_CNT'] / s_sum['TOT_PROFICIENT_CNT'] * 100
        shared_years = sorted(set(r_sum.index) & set(s_sum.index))

        fig, ax = plt.subplots(figsize=(9, 5))

        # Plot region with accommodations and add value labels
        rw_values = r_sum.loc[shared_years, 'WITH_PCT']
        ax.plot(shared_years, rw_values, marker='o', label=f'{region_label} - With')
        for i_val, value in enumerate(rw_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,10),
                        ha='center',
                        fontsize=8)

        # Plot region without accommodations and add value labels
        rwo_values = r_sum.loc[shared_years, 'WOUT_PCT']
        ax.plot(shared_years, rwo_values, marker='o', label=f'{region_label} - Without')
        for i_val, value in enumerate(rwo_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(0,-15),
                        ha='center',
                        fontsize=8)

        # Plot state with accommodations and add value labels
        sw_values = s_sum.loc[shared_years, 'WITH_PCT']
        ax.plot(shared_years, sw_values, marker='o', linestyle='--', label='State - With')
        for i_val, value in enumerate(sw_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(10,0),
                        ha='left',
                        fontsize=8)

        # Plot state without accommodations and add value labels
        swo_values = s_sum.loc[shared_years, 'WOUT_PCT']
        ax.plot(shared_years, swo_values, marker='o', linestyle='--', label='State - Without')
        for i_val, value in enumerate(swo_values):
            ax.annotate(f"{value:.1f}",
                        (shared_years[i_val], value),
                        textcoords="offset points",
                        xytext=(-10,0),
                        ha='right',
                        fontsize=8)

        ax.set_title(f"Level 4: All Grades Combined - Accommodation Status (With & Without)\n{region_label} vs. Statewide ({year_range_text})")
        ax.set_xlabel("School Year")
        ax.set_ylabel("% of Proficient Students with IEPs")
        ax.set_ylim(0, 100)
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        # Save to graphs directory (no footnote on graph)
        img_filename = f"{region_label.lower().replace(' ', '_')}_level4_combined.png"
        img_path = f"{graphs_dir}/{img_filename}"
        fig.savefig(img_path, dpi=300, bbox_inches='tight')
        plt.close()

        sheet = wb.create_sheet("Level 4 Combined")
        headers = [variable_labels.get("SCHOOL_YEAR", "School Year"),
                  f"{region_label} - With Accommodations (%)",
                  f"{region_label} - Without Accommodations (%)",
                  "State - With Accommodations (%)",
                  "State - Without Accommodations (%)"]
        sheet.append(headers)

        # Format headers professionally
        format_professional_header(sheet, 1, 1, len(headers), fonts, fills, thin_border)

        for yr in shared_years:
            sheet.append([
                yr,
                round(r_sum.loc[yr, 'WITH_PCT'], 1),
                round(r_sum.loc[yr, 'WOUT_PCT'], 1),
                round(s_sum.loc[yr, 'WITH_PCT'], 1),
                round(s_sum.loc[yr, 'WOUT_PCT'], 1)
            ])

        # Format data rows professionally
        format_data_rows(sheet, 2, len(shared_years) + 1, 1, len(headers), fonts, thin_border)

        # Auto-adjust column widths
        for col in range(1, len(headers) + 1):
            sheet.column_dimensions[get_column_letter(col)].width = 22

        # Add image with Level 4 specific sizing: Height: 4.86" (467px) Width: 8.83" (848px)
        img = XLImage(img_path)
        img.width = 848
        img.height = 467
        sheet.add_image(img, "A11")

        # Add Level4 footnote in a styled box below the data table
        footnote_row = len(shared_years) + 3  # Add some space below the data
        add_footnote_box(sheet, level_footnotes["Level4"], footnote_row, len(headers), fonts, fills, note_border)

    # Save Excel file - with lowercase and underscores
    excel_filename = f"{region_label.lower().replace(' ', '_')}_sppi_3b_multi_level_analysis.xlsx"
    excel_path = f"{output_dir}/{excel_filename}"
    wb.save(excel_path)

# Zip and download
zip_path = shutil.make_archive("sppi_3b_multi_level_analysis", 'zip', output_dir)
files.download(zip_path)

print("Analysis complete! Generated 20 Excel workbooks with professional formatting.")
print("Footnotes are now in styled boxes under the data tables in each worksheet.")

Saving sppi_3b_data.csv to sppi_3b_data.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Analysis complete! Generated 20 Excel workbooks with professional formatting.
Footnotes are now in styled boxes under the data tables in each worksheet.
